### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [ ]:
# Enter connection information and wallet file directory

database_name = "DBTENENDATA"
database_user = "Admin"
database_password = "Samiam1234567"
wallet_storage_directory = os.path.join(os.path.expanduser("~"),"ADW")b

In [ ]:
import configparser
import re
import shutil
from tempfile import NamedTemporaryFile
from zipfile import ZipFile
import os
import ads

ads.set_documentation_mode(False)

wallet_path = os.path.join(wallet_storage_directory, database_name)

# Prepare to store ADB connection information
adb_config = os.path.join(wallet_storage_directory, '.credentials')

# Write a configuration file for login creds.
config = configparser.ConfigParser()
config.read(adb_config)
config[database_name] = {'tns_admin': wallet_path,
                         'sid': '{}_medium'.format(database_name.lower()),
                         'user': database_user,
                         'password': database_password}
with open(adb_config, 'w') as configfile:
    config.write(configfile)

In [ ]:
# Read in the credentials configuration files
my_config = configparser.ConfigParser()
my_config.read(adb_config)

# Access a setting
# print(my_config[database_name].get('user'))

# Limit the information to a specific database
my_creds = my_config[database_name]
print(my_creds.get('user'))

In [ ]:
# extract the wallet
wallet_file = 'Wallet_{}.zip'.format(database_name)
wallet_filename = os.path.join(wallet_storage_directory, wallet_file)
if not os.path.exists(wallet_filename):
    print("The file {} does not exist.".format(wallet_filename))
    print("Please copy the Wallet file, {}, into the directory {} then rerun this cell.".format(wallet_file, wallet_filename))
else:
    os.makedirs(wallet_path, mode=0o700, exist_ok=True)
    with ZipFile(wallet_filename, 'r') as zipObj:
        zipObj.extractall(wallet_path)

In [ ]:
# Update the sqlnet.ora

sqlnet_path = os.path.join(wallet_path, 'sqlnet.ora')
sqlnet_original_path = os.path.join(wallet_path, 'sqlnet.ora.original')
sqlnet_backup_path = os.path.join(wallet_path, 'sqlnet.ora.backup')
if not os.path.exists(sqlnet_original_path):
    shutil.copy(sqlnet_path, sqlnet_original_path)
if os.path.exists(sqlnet_path):
    shutil.copy(sqlnet_path, sqlnet_backup_path)
sqlnet_re = re.compile('(WALLET_LOCATION\s*=.*METHOD_DATA\s*=.*DIRECTORY\s*=\s*\")(.*)(\".*)', 
                       re.IGNORECASE)
tmp = NamedTemporaryFile()
with open(sqlnet_path, 'rt') as sqlnet:
    for line in sqlnet:
        tmp.write(bytearray(sqlnet_re.subn(r'\1{}\3'.format(wallet_path), line)[0], 
                            encoding='utf-8'))
tmp.flush()
shutil.copy(tmp.name, sqlnet_path)
tmp.close()

In [ ]:
# Add TNS_ADMIN to the environment
os.environ['TNS_ADMIN'] = config[database_name].get('tns_admin')

# Test the database connection
creds = config[database_name]
connect = 'sqlplus ' + creds.get('user') + '/' + creds.get('password') + '@' + creds.get('sid')
print(os.popen(connect).read())

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import pandas as pd
import logging

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

In [ ]:
# Add TNS_ADMIN to the environment
#creds = config[database_name]
#os.environ['TNS_ADMIN'] = config[database_name].get('tns_admin')

# Get the URI to connect to the database
uri='oracle+cx_oracle://' + creds.get('user') + ':' + creds.get('password') + '@' + creds.get('sid')
print(uri)